In [ ]:
#inititalize the notebook
import os
import textwrap
import pymongo
from dotenv import load_dotenv
from openai import AzureOpenAI

load_dotenv()

openai_client = AzureOpenAI(
    api_key = os.getenv("AZURE_OPENAI_API_KEY"),
    api_version = os.getenv("OPENAI_API_VERSION"),
    azure_endpoint = os.getenv("AZURE_OPENAI_API_BASE")
)

mongo_user = os.getenv("COSMOS_MONGO_USER")
mongo_pwd = os.getenv("COSMOS_MONGO_PWD")
mongo_server = os.getenv("COSMOS_MONGO_SERVER")
mongo_conn = f"mongodb+srv://{mongo_user}:{mongo_pwd}@{mongo_server}?tls=true&authMechanism=SCRAM-SHA-256&retrywrites=false&maxIdleTimeMS=120000"


embeddings_deployment_name = "text-embedding-ada-002"
chat_deployment_name = "gpt-4o"

print("*** init done! ***")


Hoe ziet een vector er uit?

In [ ]:
word = "cat"

embeddings = openai_client.embeddings.create(input = word, model=embeddings_deployment_name)

embeddingData = embeddings.data[0].embedding

print("De vector voor de het woord: " + word)
print("Het aantal dimensies in de vector is: " + str(len(embeddingData)))
print("Dit zijn de waarden van de vector: ")
print(embeddingData)

Vergelijk de semantische gelijkheid van woorden

We hebben een lijst van woorden en een woord dat we willen vergelijken.

* Bereken een vector van ieder woord in de lijst
* Bereken een vector van het woord dat we willen vergelijken
* Bepaal de gelijkheid (score) met een dotproduct algoritme (er zijn meerdere algoritmes om dit te doen)

In [ ]:
### DotProduct is a simple algorithm to calculate simularity between two vectors
### Multiply each element in vector1 with the corresponding element in vector2
### Add the results together for each pair of elements
### The result is the dot product of the two vectors
### https://www.mathsisfun.com/algebra/vectors-dot-product.html

def dotproduct(vector1, vector2):
    #return sum((a*b) for a, b in zip(vector1, vector2))
    #get the length of the vector1 
    length = len(vector1)

    #check if the length of the vector2 is the same as vector1
    if length != len(vector2):
        print("The vectors are not the same length")
        return

    #iterate over the vector based on the length of the vector
    sum = 0

    for i in range(length):
        #multiply the vectors and add them to the sum
        sum += vector1[i] * vector2[i]

    return sum

##### end of the dotproduct function #####

words = ['cat', 
         'hamburger', 
         'computer', 
         'server',
         'dog', 
         'pizza', 
         'laptop', 
         'horse', 
         'car',
         'steak', 
         'truck',
         'pickup']

word = 'food'

#create a list of embeddings for each word
wordListEmbedding = []
for w in words:
    response = openai_client.embeddings.create(input = w, model=embeddings_deployment_name).data[0].embedding
    wordListEmbedding.append(response)

#create an embedding for the word we want to compare
wordEmbedding = openai_client.embeddings.create(input = word, model=embeddings_deployment_name).data[0].embedding

#calculate the dot product between the word we want to compare and each word in the list
dotProductList = []
for w in wordListEmbedding:
    dotProductList.append(dotproduct(wordEmbedding, w))

#sort the dotProductList
sortedDotProductList = sorted(dotProductList, reverse=True)

#find the index of the words with the highest dot product
indexList = []
for i in sortedDotProductList:
    indexList.append(dotProductList.index(i))

#display the top 3 words based on indexlist
print("De top 3 woorden vergelijkbaar met '" + word + "' zijn: ")
print(f"'{words[indexList[0]]}' dotproduct score: + {str(sortedDotProductList[0])}")
print(f"'{words[indexList[1]]}' dotproduct score: + {str(sortedDotProductList[1])}")
print(f"'{words[indexList[2]]}' dotproduct score: + {str(sortedDotProductList[2])}")






De vector kun je ook berekenen over een zin.

In dit geval nemen we user prompt uit het laatste voorbeeld dat we hebben gezien bij "prompts"

In [ ]:
sentence = "wanneer is de informatieavond?"

embeddings = openai_client.embeddings.create(input = sentence, model=embeddings_deployment_name)

embeddingData = embeddings.data[0].embedding

print("De vector voor de de zin: " + sentence)
print("Het aantal dimensies in de vector is: " + str(len(embeddingData)))
print("Dit zijn de waardes van de vector: ")
print(embeddingData)

laten we eens kijken of we de meest relevante pagina kunnen vinden om de vraag te beantwoorden.

* We gaan een vector maken van de vraag
* we gaan voor iedere pagina een vector maken
* We gaan de meest relevante pagina selecteren en in de prompt meegeven

In [ ]:
#lees de pagina's van de website
page1 = open("../pages/home.md", "r").read()
page2 = open("../pages/informatieavond-5-november.md", "r").read()

pages = [page1, page2]

myDataQuestion = "wanneer is de informatieavond?"

pagesListEmbedding = []
for p in pages:
    response = openai_client.embeddings.create(input = p, model=embeddings_deployment_name).data[0].embedding
    pagesListEmbedding.append(response)

#create an embedding for the word we want to compare
questionEmbedding = openai_client.embeddings.create(input = myDataQuestion, model=embeddings_deployment_name).data[0].embedding

#calculate the dot product between the word we want to compare and each word in the list
dotProductList = []
for p in pagesListEmbedding:
    dotProductList.append(dotproduct(questionEmbedding, p))

#sort the dotProductList
sortedDotProductList = sorted(dotProductList, reverse=True)

#find the index of the words with the highest dot product
indexList = []
for i in sortedDotProductList:
    indexList.append(dotProductList.index(i))

#augment the prompt with the most relevant page
myDataResponse = openai_client.chat.completions.create(
    model = chat_deployment_name,
    temperature=0.0,
    messages=[
        {
            "role": "user",
            "content": myDataQuestion
        },
        {
            "role": "system",
            "content" : f"""
                Je bent een behulpzame assistent
                Je geeft alleen antwoord op basis van de data die je hebt gekregen
                Indien je het antwoord niet kunt vinden, 
                zeg je dat je het antwoord niet weet
                
                ## data
                {pages[indexList[0]]}

                """
        }
    ]
)


lines = textwrap.wrap(myDataResponse.choices[0].message.content, width=80)
for line in lines:
    print(line)

print("----")
print("Hier wat info over de tokens die je hebt gebruikt:")
print(myDataResponse.usage)



Dit is al een stuk beter!

Maar hebben we het hele document wel nodig?

In [ ]:
# create an array of files in the chunks directory
files = os.listdir("../chunks")
pages = []

for file in files:
    page = open(f"../chunks/{file}", "r").read()
    pages.append(page)

pagesListEmbedding = []
for p in pages:
    response = openai_client.embeddings.create(input = p, model=embeddings_deployment_name).data[0].embedding
    pagesListEmbedding.append(response)

#create an embedding for the word we want to compare
questionEmbedding = openai_client.embeddings.create(input = myDataQuestion, model=embeddings_deployment_name).data[0].embedding

#calculate the dot product between the word we want to compare and each word in the list
dotProductList = []
for p in pagesListEmbedding:
    dotProductList.append(dotproduct(questionEmbedding, p))

#sort the dotProductList
sortedDotProductList = sorted(dotProductList, reverse=True)

#find the index of the words with the highest dot product
indexList = []
for i in sortedDotProductList:
    indexList.append(dotProductList.index(i))

#augment the prompt with the most relevant page
myDataResponse = openai_client.chat.completions.create(
    model = chat_deployment_name,
    temperature=0.0,
    messages=[
        {
            "role": "user",
            "content": myDataQuestion
        },
        {
            "role": "system",
            "content" : f"""
                Je bent een behulpzame assistent
                Je geeft alleen antwoord op basis van de data die je hebt gekregen
                Indien je het antwoord niet kunt vinden, 
                zeg je dat je het antwoord niet weet
                
                ## data
                {pages[indexList[0]]}

                """
        }
    ]
)


lines = textwrap.wrap(myDataResponse.choices[0].message.content, width=80)
for line in lines:
    print(line)

print("----")
print("Hier wat info over de tokens die je hebt gebruikt:")
print(myDataResponse.usage)



Weer een verbetering! Minder tokens en toch een antwoord!

Embeddings berekenen kost echter ook tokens

In [ ]:
response = openai_client.embeddings.create(input = pages[indexList[0]], model=embeddings_deployment_name)
response.usage

Vectoren berekenen hoeven we niet iedere keer te doen.

We kunnen de vectoren ook opslaan en hergebruiken.

Hier hebben we een vector database voor nodig. 

Er zijn verschillende vector databases. 

We kiezen hier voor MongoDb. In dit geval gebruiken Cosine Simularity als algoritme. Dit algoritme is ingebouwd in de database om gelijkheid te berekenen.

In [ ]:
databaseName = "vectorsample"
collectionName = "chunks"

mongo_client = pymongo.MongoClient(mongo_conn)
db = mongo_client[databaseName]
collection = db[collectionName]

if collectionName not in db.list_collection_names():
    db.create_collection(collectionName)
else:
    #delete all documents in the collection so we can rerun this script
    collection.delete_many({})


db.command({
  'createIndexes': f'{collectionName}',
  'indexes': [
    {
      'name': 'VectorSearchIndex',
      'key': {
        "contentVector": "cosmosSearch"
      },
      'cosmosSearchOptions': {
        'kind': 'vector-ivf',
        'numLists': 1,
        'similarity': 'COS',
        'dimensions': 1536
      }
    }
  ]
})

files = os.listdir("../chunks")
pages = []
filesUploaded = 0
for file in files:
    page = open(f"../chunks/{file}", "r").read()
    response = openai_client.embeddings.create(input = page, model=embeddings_deployment_name).data[0].embedding
    
    filetoStore = {
        "filename": file,
        "content": page,
        "contentVector": response
    }

    collection.insert_one(filetoStore)
    filesUploaded += 1

print(f"{filesUploaded} bestanden en vectoren opgeslagen in de database")


We kunnen nu een database query uitvoeren met de vector van de vraag als input (dit is geen SQL query, maar een vector search opdracht).

De output is een lijst met relevante documenten die in de database zijn opgeslagen.

In [ ]:
def vector_search(vector, num_results):
    pipeline = [
        {
            '$search': {
                "cosmosSearch": {
                    "vector": vector,
                    "path": "contentVector",
                    "k": num_results 
                },
                "returnStoredSource": True }},
        {'$project': { 'similarityScore': { '$meta': 'searchScore' }, 'document' : '$$ROOT' } }
    ]
    results = collection.aggregate(pipeline)
    return results

myDataQuestion = "wanneer is de informatieavond?"
#create an embedding for the question we want to compare
questionEmbedding = openai_client.embeddings.create(input = myDataQuestion, model=embeddings_deployment_name).data[0].embedding

#zoek 3 relevante pagina's op basis van de vraag
results = vector_search(questionEmbedding, 3)

print("De top 3 resultaten zijn: ")
for result in results:
    print(result["document"]["filename"] + " - " + str(result["similarityScore"]))
